In [1]:
import requests
from api_keys import google_maps_key
from api_keys import yelp_key

# Set up the Yelp API endpoint URL
base_url = 'https://api.yelp.com/v3/businesses/search'

# Set up the Google Geocoding API endpoint URL
geocoding_url = 'https://maps.googleapis.com/maps/api/geocode/json'

In [12]:
def fetch_restaurants(latitude, longitude):
    headers = {
        'Authorization': f'Bearer {yelp_key}'
    }

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': 1000,
        'categories': 'restaurants',
        'limit': 50
    }

    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        restaurant_data = response.json()
        return restaurant_data.get('businesses', [])
    else:
        print('Error occurred while fetching restaurant data.')
        return []


In [13]:
address = input('Enter an address: ')
#2156 North Claremont Ave, Chicago, IL 60647

Enter an address: 2156 North Claremont Ave, Chicago, IL 60647


In [14]:
params = {
    'address': address,
    'key': google_maps_key
}

response = requests.get(geocoding_url, params=params)

if response.status_code == 200:
    geocoding_data = response.json()

    results = geocoding_data.get('results', [])

    if results:
        location = results[0].get('geometry', {}).get('location')
        latitude = location.get('lat')
        longitude = location.get('lng')

        restaurants = fetch_restaurants(latitude, longitude)

        if restaurants:
            print(f'Total number of restaurants: {len(restaurants)}')

            restaurant_counts = {}
            for restaurant in restaurants:
                category = restaurant.get('categories', [{}])[0].get('title')
                restaurant_counts[category] = restaurant_counts.get(category, 0) + 1

            print('Restaurant counts by category:')
            for category, count in restaurant_counts.items():
                print(f'{category}: {count}')
        else:
            print('No restaurants found within the specified radius.')
    else:
        print('No results found for the given address.')
else:
    print('Error occurred while geocoding the address.')

Total number of restaurants: 50
Restaurant counts by category:
Cocktail Bars: 2
Ramen: 1
Mexican: 5
Malaysian: 1
Tacos: 1
Italian: 3
Cuban: 3
Pizza: 3
Latin American: 3
Burgers: 1
Poke: 1
Bars: 2
French: 1
Breakfast & Brunch: 2
Modern European: 1
Breweries: 1
American (New): 2
Korean: 1
Thai: 2
Barbeque: 1
Fast Food: 1
Sushi Bars: 4
Gastropubs: 1
Steakhouses: 1
Chocolatiers & Shops: 1
Hot Dogs: 1
Coffee & Tea: 1
Indian: 1
Cafes: 2


In [5]:
# def fetch_restaurants(latitude, longitude):
#     headers = {
#         'Authorization': f'Bearer {yelp_key}'
#     }

#     # Set the query parameters for the Yelp API request
#     params = {
#         'latitude': latitude,
#         'longitude': longitude,
#         'radius': 1000,  # 1 km radius
#         'categories': 'restaurants'
#     }

#     # Send the Yelp API request
#     response = requests.get(base_url, headers=headers, params=params)

#     # Check if the request was successful
#     if response.status_code == 200:
#         restaurant_data = response.json()
#         return restaurant_data.get('businesses', [])
#     else:
#         print('Error occurred while fetching restaurant data.')
#         return []

# # Prompt the user for latitude and longitude
# latitude = float(input('Enter latitude: '))
# longitude = float(input('Enter longitude: '))

# restaurants = fetch_restaurants(latitude, longitude)

# if restaurants:
#     # Display restaurant information
#     print(f'Total number of restaurants: {len(restaurants)}')

#     # Count the types of restaurants
#     restaurant_counts = {}
#     for restaurant in restaurants:
#         category = restaurant.get('categories', [{}])[0].get('title')
#         restaurant_counts[category] = restaurant_counts.get(category, 0) + 1

#     print('Restaurant counts by category:')
#     for category, count in restaurant_counts.items():
#         print(f'{category}: {count}')
# else:
#     print('No restaurants found within the specified radius.')
#41.92166722859687, -87.68671808006695

Enter latitude: 41.92166722859687
Enter longitude: -87.68671808006695
Total number of restaurants: 20
Restaurant counts by category:
Cocktail Bars: 1
Ramen: 1
Mexican: 2
Malaysian: 1
Tacos: 1
Italian: 1
Pizza: 2
Latin American: 1
Burgers: 1
Cuban: 1
Bars: 2
Breakfast & Brunch: 1
Modern European: 1
French: 1
Poke: 1
Breweries: 1
American (New): 1
